<a href="https://colab.research.google.com/github/chenbrilliancesol/machine-learning/blob/main/%E5%9F%BA%E4%BA%8Esklearn%E7%9A%84GradientBoostingClassifier%E8%BF%9B%E8%A1%8C%E7%9A%84%E6%B3%B0%E5%9D%A6%E5%B0%BC%E5%85%8B%E5%8F%B7%E7%94%9F%E5%AD%98%E9%A2%84%E6%B5%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [2]:
train=pd.read_csv('/content/train.csv')
test=pd.read_csv('/content/test.csv')

In [5]:
print(train.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [9]:
def quick_feat(df):
  df=df.copy()
  df['Age']=df['Age'].fillna(df['Age'].median())
  df['Embarked']=df['Embarked'].fillna(df['Embarked'].mode()[0])
  df['Fare']=df['Fare'].fillna(df['Fare'].median())
  df['Family']=df['SibSp']+df['Parch']+1
  df['IsAlone']=(df['Family']==1).astype(int)
  df['Title']=df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
  df['Title']=df['Title'].replace(['Lady','Countess','Capt','Col','Don','Dr','Major','Rev','Sir','Jonkheer','Dona'],'Rare')
  df['Title']=df['Title'].replace('Mlle','Miss').replace('Ms','Miss').replace('Mme','Mrs')
  df['AgeBin']=pd.cut(df['Age'],bins=[0,12,20,40,60,80], labels=[0,1,2,3,4])
  df['FareBin']=pd.qcut(df['Fare'],4,labels=[0,1,2,3])
  return df

In [8]:
print(train.dtypes)

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


In [10]:
train=quick_feat(train)
test=quick_feat(test)

In [11]:
num_cols = ['Age','Fare','Family','Pclass','SibSp','Parch']
cat_cols = ['Sex','Embarked','Title','IsAlone','AgeBin','FareBin']
X = train[num_cols + cat_cols]
y = train['Survived']
X_test = test[num_cols + cat_cols]

In [12]:
preprocess = ColumnTransformer([
        ('num', SimpleImputer(strategy='median'), num_cols),
        ('cat', Pipeline([('imp', SimpleImputer(strategy='most_frequent')),
        ('ohe', OneHotEncoder(handle_unknown='ignore'))]),cat_cols)
        ])
clf = GradientBoostingClassifier(random_state=42, learning_rate=0.05,n_estimators=2500, max_depth=3, subsample=0.8)
pipe = Pipeline([('prep', preprocess),('model', clf)])
pipe.fit(X,y)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='median'),
                                                  ['Age', 'Fare', 'Family',
                                                   'Pclass', 'SibSp',
                                                   'Parch']),
                                                 ('cat',
                                                  Pipeline(steps=[('imp',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked', 'Title',
                                                   'IsAlone', 'AgeBin',
                                                   'FareBin'])])),
                ('model',
                 GradientBoostingClassifier(learning_rate=0.05,
                                            n_estimators=2500, random_state=42,
                                            subsample=0.8))])

In [13]:
pred = pipe.predict(X_test)
sub = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': pred})
sub.to_csv('submission_gb_top2p.csv', index=False)
print('submission saved!')

submission saved!


In [14]:
pred_train = pipe.predict(X)
from sklearn.metrics import accuracy_score
acc = accuracy_score(y, pred_train)
print(f"训练集准确度 (Accuracy) = {acc:.4f}")
preview = pd.DataFrame({
    'PassengerId': train['PassengerId'],
    'Actual': y,
    'Predicted': pred_train
})
print(preview.head(20))

训练集准确度 (Accuracy) = 0.9820
    PassengerId  Actual  Predicted
0             1       0          0
1             2       1          1
2             3       1          1
3             4       1          1
4             5       0          0
5             6       0          0
6             7       0          0
7             8       0          0
8             9       1          1
9            10       1          1
10           11       1          1
11           12       1          1
12           13       0          0
13           14       0          0
14           15       0          0
15           16       1          1
16           17       0          0
17           18       1          0
18           19       0          0
19           20       1          1
